In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from time import perf_counter
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# INF554 Lab 4: Spectral Clustering and Feature Selection


In this lab we will add the spectral clustering algorithm to our collection on unsupervised learning techniques, which we started to build in Lab 3. We will furthermore explore the $\chi^2$ measure for feature selection in a supervised classification task. 

## Spectral Clustering

We begin by briefly reviewing the spectral clustering algorithm. For more detail on this algorithm please revisit the slides from lecture 3.

In the spectral clustering algorithm we utilise the spectral decomposition of a graph representation matrix such as the random walk Laplacian $L_{rw} = I - D^{-1}A$ to cluster a graph or network. Specifically, the spectral clustering algorithm consists of two steps. First we compute the spectral decomposition of the used graph representation matrix,
$$
L_{rw} = U \Lambda U^T.
$$
Then, in the second step we run the $k$-means algorithm on the rows of the eigenvector matrix $U_k,$ containing the eigenvectors corresponding to the $k$ smallest eigenvalues of $L_{rw},$ to cluster the nodes in our graph.

If instead of a graph we only have a set of datapoints then we are still able to use the spectral clustering algorithm by utilising one of several heuristics to infer a graph from the dataset. One such method produces a weighted *fully connected graph by using the Gaussian kernel* to represent edge weights in our inferred graph. Another is to infer a $k$*-nearest neighbours graph*, where two nodes are connected if either of them are in within the $k$ nearest neighbours of the other in Euclidean distance.



### Dataset: Three Concentric Circles

We begin by generating a variation of the famous synthetic, concentric circles dataset.



In [ ]:
np.random.seed(5)

def get_circle(r=1.0, N=150):  
    # Use polar coords to get unif dist points  
    step = np.pi * 2.0 / N  
    t = np.arange(0, np.pi * 2.0, step)  
    x_1 = r * np.cos(t)  
    x_2 = r * np.sin(t)  
    return np.column_stack((x_1, x_2))
      
def get_noise(stddev=0.2, N=150):  
    # 2d gaussian random noise  
    x_1 = np.random.normal(0, stddev, N)  
    x_2 = np.random.normal(0, stddev, N)  
    return np.column_stack((x_1, x_2))    
  
def generateData(sigma):
    inner_circle = get_circle(r=1, N=50) + get_noise(sigma,N=50)
    middle_circle = get_circle(r=4, N=400) + get_noise(sigma,N=400)
    outer_circle = get_circle(r=6, N=800) + get_noise(sigma, N=800)
    return np.vstack([inner_circle, middle_circle, outer_circle])
 
 
X_low_var = generateData(0.2)

X_high_var = generateData(0.3)



# Plot data
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].scatter(X_low_var[:,0], X_low_var[:,1])
ax[0].title.set_text('Dataset low variance')
ax[1].scatter(X_high_var[:,0], X_high_var[:,1])
ax[1].title.set_text('Dataset high variance')
plt.show()

>**Task 1:** Use the [scikit-learn kmeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) implementation to cluster the two datasets using the kmeans algorithm (Note that we already loaded the KMeans function for you in code cell 1). Please store the produced labels in the two variables ```labels_low_var``` and ```labels_high_var```. You may assume knowledge of the true number of clusters, i.e., k=3.

In [ ]:
labels_low_var = np.zeros(X_low_var.shape[0])
labels_high_var = np.zeros(X_high_var.shape[0])

k = 3

#Please insert your code for Task 1 here



# Plot data
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].scatter(X_low_var[:,0], X_low_var[:,1], c=labels_low_var, facecolors="none") 
ax[0].title.set_text('$k$-Means Clustering')

ax[1].scatter(X_high_var[:,0], X_high_var[:,1], c=labels_high_var, facecolors="none") 
ax[1].title.set_text('$k$-Means Clustering')

plt.show()

>**Task 2:** Implement the function ```knn_graph``` to construct a k-nearest neighbour graph from a set of data points. Furthermore, fill out the ```gaussianKernel``` function to infer a weighted fully connected graph from the two datasets using the Gaussian kernel as a similarity function. (The Gaussian kernel will be discussed in greater detail in Lab 5, for now it suffices to know that it contains a free parameter $\sigma$ and is defined as follows $  K_{Gauss}(x_i, x_j) = \exp\left( - \frac{\| x_i - x_j\|^2}{2 \sigma^2}\right).$)

In [ ]:
def knn_graph(X, neighbours):
    """
    Args:
        X (np.array, n x p):  data points
        neighbours (int): number of nearest neighbours to be considered for each node
    
    Returns:
        A (np.array, n x n): the adjacency matrix of the knn graph
    """    
    
    n = X.shape[0]
    A = np.zeros((n,n))
    
    #Please insert your code for Task 2 here        
    
        
    return A


def gaussianKernel(X1, X2, sigma = 0.1):
    """
    Args:
        X1 (np.array, n_1 x p): a set of n_1 datapoints to be used as the first of the two kernel inputs
        X2 (np.array, n_2 x p): a set of n_2 datapoints to be used as the second of the two kernel inputs
    
    Returns:
        K (np.array, n_1 x n_2): a kernel matrix containing the kernel values measuring of all possible data point pairs
    """    
    K = np.zeros((X1.shape[0],X2.shape[0]))
     
    #Please insert your code for Task 2 here

    
    return K


A_low_var_Gaus = gaussianKernel(X_low_var, X_low_var)
A_high_var_Gaus = gaussianKernel(X_high_var, X_high_var)


neighbours = 8
A_low_var_knn = knn_graph(X_low_var, neighbours)
A_high_var_knn =  knn_graph(X_high_var, neighbours)


    
plt.figure(1, figsize=(15,15))
G_high_var = nx.from_numpy_matrix(A_high_var_knn-np.eye(A_high_var_knn.shape[0]))
nx.draw_networkx(G_high_var, pos=X_high_var, node_size=10)
plt.title("%d-nearest neighbour graph for the high variance concentric circles (with self-loops removed)" %(neighbours))
plt.show()


>**Task 3:** Implement the spectral clustering algorithm and use it to cluster the four graphs obtained in Task 2.

In [ ]:
def spectral_clustering(A,k):
    """
    Args:
        A (np.array, n x n): the adjacency matrix of our graph
        k (int): the number of clusters
    
    Returns:
        classes (np.array, n): the inferred class labels of the n nodes
        U_k (np.array, nxk): the k eigenvectors used in the 
    """    
    #Please insert your code for Task 3 here

    
    return classes, U_k




classes_low_var_Gaus, U_k_low_var_Gaus = spectral_clustering(A_low_var_Gaus,3)
classes_high_var_Gaus, U_k_high_var_Gaus = spectral_clustering(A_high_var_Gaus,3)

classes_low_var_knn, U_k_low_var_knn = spectral_clustering(A_low_var_knn,3)
classes_high_var_knn, U_k_high_var_knn = spectral_clustering(A_high_var_knn,3)


fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,15))
ax[0,0].scatter(X_low_var[:,0], X_low_var[:,1], c=classes_low_var_Gaus, facecolors="none") 
ax[0,0].title.set_text('Spectral Clustering on the Gaussian similarity graph')

ax[0,1].scatter(X_high_var[:,0], X_high_var[:,1], c=classes_high_var_Gaus, facecolors="none") 
ax[0,1].title.set_text('Spectral Clustering on the Gaussian similarity graph')

ax[1,0].scatter(X_low_var[:,0], X_low_var[:,1], c=classes_low_var_knn, facecolors="none") 
ax[1,0].title.set_text('Spectral Clustering on the knn graph')

ax[1,1].scatter(X_high_var[:,0], X_high_var[:,1], c=classes_high_var_knn, facecolors="none") 
ax[1,1].title.set_text('Spectral Clustering on the knn graph')

plt.show()




## 2) Feature selection using the $\chi^2$ measure


We will now implement and apply the $\chi^2$ measure to examine its performance in a supervised  classification task. We begin by describing and loading the considered data set.



### Dataset

The dataset  describes a set of $102$ molecules of which $39$ are judged by human experts to be musks (class 1) and the remaining 63 molecules are judged to be non-musks (class 2). The goal is to learn to predict whether new molecules will be musks or non-musks, a binary classification task. However, the $166$ features that describe these molecules depend upon the exact shape, or conformation, of the molecule. Because bonds can rotate, a single molecule can adopt many different shapes. To generate this data set, all the low-energy conformations of the molecules were generated to produce $6598$ conformations. Then, a feature vector was extracted that describes each conformation. The final dataset has $6598$ instances and $166$ features.


In [ ]:
# Loading Data
data = np.loadtxt('data/musk.csv', delimiter=',')
Y = data[:,0:1].reshape(-1)
X = data[:,1:data.shape[1]]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=5)

print('This is the first datapoint: \n', X_train[0,:])

We will use a logistic regression classifier (which we saw in Lab 2) to classify our molecules. Recall that the predictions produced by the logistic regression classifier are not binary, but in the range $[0,1]$. Hence, we are able to choose a threshold above which we assign the label musk and below which we classify the molecule as non-musk. In order to assess our classifier independent of our chosen threshold we use $101$ different threshold values (from $0$ to $1$ with steps of size $0.01$) to assign class labels. This allows us to plot a *Receiver Operating Characteristic (ROC)* curve to visually compare the quality of our different classifiers. Recall from Lecture 4 that, the ROC curve is produced by plotting the true positive rate (TPR) on the $y$-axis against the false positive rate (FPR) on the $x$-axis as the  threshold varies, where
\begin{align*}
\text{TPR} &= \frac{\text{true positive}}{\text{true positive}+{\text{false negative}}} \\
\text{FPR} &= \frac{\text{false positve}}{\text{false positive}+{\text{true negative}}}.
\end{align*}

ROC curves are evaluated by observing the *Area Under the ROC Curve (AUC).* Models with a large AUC are generally preferable over those with a low AUC.

> **Task 4:** Complete the below ```roc_auc``` function by making predictions from the logistic regression model based on the threshold set in the current iteration, calculating the corresponding true (```tpr```), false positive rates (```fpr```) and use these to approximate the area under the ROC Curve (```auc```). This will allow you to use the provided code plotting the ROC curve of the logistic regression classifier applied to the musk dataset with all features present.


In [ ]:
def roc_auc(model, X, Y):
    """
    Args:
        model: a trained model
        X (np.array, n x p): data
        Y (np.array, n): target
    
    Returns:
        tpr (np.array, 101): vector containing the true positive rates corresponding 
                             to the different classification thresholds
        fpr (np.array, 101): vector containing the false positive rates corresponding 
                             to the different classification thresholds
        auc (float): Area Under the ROC Curve
    """
    probs = model.predict_proba(X)
    
    
    num_thresholds = 101
    tpr = np.zeros(num_thresholds)
    fpr = np.zeros(num_thresholds)
    for i in np.arange(101):
        threshold = i/100

        # Please insert the code for Task 4 here 
        

        
    
    return tpr, fpr, auc


model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

res = model.predict(X_test) #Note this uses the default threshold set in scikit-learn equal to 0.5
tpr, fpr, auc = roc_auc(model, X_test, y_test.astype("int"))
plt.figure(figsize=(10,7))
plt.plot(fpr, tpr)
plt.title("Acc = %.2f, AUC = %.2f" % (np.sum(res==y_test)/len(res), auc))
plt.xlabel("FPR ")
plt.ylabel("TPR ")
plt.show()


Now we are ready to consider the $\chi^2$ measure for feature selection. We work with a data set of $n$ data points $\{X_i\}_{i=1}^n,$ where each data point is $p$-dimensional, i.e., we have $p$ features, denoted $X_{\cdot j}$ for $j \in \{1, \ldots, p\},$ in our dataset. We furthermore assume that we are working with discrete data where $x_{ij}\in \{1, \ldots, V\}$ for all $i \in \{1, \dots, n\}$ and $j\in \{1, \ldots, p\}.$ Further denote our discrete dependent variable by $Y$ containing class labels of our data points, i.e., $y_i\in \{1, \ldots, C\}.$ Assume that the features in our dataset $X_{\cdot j}$ are sampled from the distribution of a random variable $\mathcal{X}_j$ and the labels in our dataset $Y$ are sampled from the distribution of a random variable $\mathcal{Y}.$ Then, the $\chi^2$ measure is defined in terms of the following two quantities
\begin{equation*}
	O_{vc} = \sum_{i=1}^n \Big(I(x_{ij} = v) \cdot I(y_i = c)\Big), 
\end{equation*}
which is used to estimate $n \cdot P(\mathcal{X}_{j}=v,\mathcal{Y}=c),$ i.e., the joint probability of feature $\mathcal{X}_j$ taking value $v$ and $\mathcal{Y}$ taking value c, and 
 \begin{equation*}
	E_{vc} =  \frac{1}{n}\sum_{i=1}^n I(x_{ij} = v) \cdot \sum_{i=1}^n I(y_i = c), 
\end{equation*}
which estimates the probability $n \cdot P(\mathcal{X}_j =v) \cdot P(\mathcal{Y}=c),$ i.e., the joint probability of feature $\mathcal{X}_j$ taking value $v$ and $\mathcal{Y}$ taking value c if $\mathcal{X}$ and $\mathcal{Y}$ are distributed independently.  Now the $\chi^2$ measure can be defined as  
\begin{equation*}
\chi^2(X_{\cdot j},Y) = \sum_{v \in \{1, \ldots V\}} \sum_{c \in \{1, \ldots C\}} \frac{(O_{vc} - E_{vc})^2}{E_{vc}}.
\end{equation*}



In the case of independence of our data feature $\mathcal{X}_j$ and the class labels $\mathcal{Y}$, $O_{vc}$ and $E_{vc}$ should take similar values and we obtain a small $\chi^2$ measure.
If a feature $X_{\cdot j }$ is close to independent from the labels $Y$ which we want to predict from $X_{\cdot j },$ then the feature $X_{\cdot j }$ is not of great help in our learning task. 
Therefore, small values of the $\chi^2$ measure are understood to indicate that feature $X_{\cdot j }$ could  be removed without a significant accuracy loss of our classifier. 



> **Task 5:** Complete the function ```chiSQ``` to implement the $\chi^2$ measure for feature selection.

In [ ]:
def chiSQ(X, y):
    """
    Args:
        X (np.array, n x p): features
        y (np.array, n): target
        
    Returns:
        chisq (np.array, p): The ChiSQ mesure for every feature
    """
    
    
    n = X.shape[0]
    dim  = X.shape[1]
    chisq = np.zeros(dim)
    
    # Please insert the code for Task 5 here 

    
    
        
    return chisq


print('The chi squared measure of the first 5 features is\n',chiSQ(X_train[:,:5],y_train))



With these implementations we are now able to perform feature selection using the $\chi^2$ measure to select $k$ features to be used for classifcation with $k \in \{20,40,60, 80,100,150\}.$ This allows us to examine the accuracy, AUC and training time values for different values of $k.$ 

In [ ]:
k = [20, 40, 60, 80, 100, 150]

acc = np.zeros(len(k))
auc = np.zeros(len(k))
time = np.zeros(len(k))

fpr = np.zeros((len(k),101))
tpr = np.zeros((len(k),101))



chisq_values = chiSQ(X,Y)
feature_ranking = np.argsort(chisq_values)[::-1]

for i, n_features in enumerate(k):
    model = LogisticRegression(max_iter=10000)
    X_k_train = X_train[:, feature_ranking[:n_features]]
    
    start = perf_counter()
    model.fit(X_k_train, y_train)
    end = perf_counter()
    
    time[i] = end - start
    
    X_k_test = X_test[:, feature_ranking[:n_features]]
    res = model.predict(X_k_test)
    
    acc[i] = np.sum(res == y_test)/len(y_test)
    
    
    tpr[i], fpr[i], auc[i] = roc_auc(model, X_k_test, y_test.astype('int'))
    



fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(17,7))
for i in range(6):  
    ax[int(i>2), i%3].plot(fpr[i,:], tpr[i,:])
    ax[int(i>2), i%3].set_title("Top %i features, Acc = %.2f, Auc = %.2f, time= %.2f"\
                                % (k[i], acc[i], auc[i], time[i]))
    
plt.show()